<a href="https://colab.research.google.com/github/gabrielanatalia/Projects/blob/main/Portfolio_Visualizer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Portfolio Visualizer**



In [2]:
# @title Load libraries and functions
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from datetime import date, timedelta
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import fsolve
import itertools
import plotly.graph_objects as go
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from matplotlib import colors
import statsmodels.formula.api as smf
from dateutil.parser import parse
import pytz
import re
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

!git clone https://github.com/gabrielanatalia/Projects/
%cd /content/Projects
import sys
sys.path.append('/content/Projects')
import port_cons as pc

import warnings
warnings.filterwarnings("ignore")

def convert_to_datetime(input_str, parserinfo=None):
    return parse(input_str, parserinfo=parserinfo)

TOLERANCE = 1e-10

def _allocation_risk(weights, covariances):

    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]

    return portfolio_risk

def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T) \
        / portfolio_risk

    return assets_risk_contribution

def _risk_budget_objective_error(weights, args):
    covariances = args[0]
    assets_risk_budget = args[1]
    weights = np.matrix(weights)

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = \
        _assets_risk_contribution_to_allocation_risk(weights, covariances)

    assets_risk_target = \
        np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))

    error = sum(np.absolute(assets_risk_contribution - assets_risk_target.T))[0, 0]
    return error

def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},{'type': 'ineq', 'fun': lambda x: x})

    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    weights = optimize_result.x
    print(optimize_result.message)
    return weights

def rebal_wgt_riskparity(returns_data, start_date, end_date, rebal_months=[4,10], halflife=3.5, annualized=252, shrink_covar=False):
    num_of_assets = len(returns_data.columns)

    ret_data_filtered = returns_data.loc[start_date:end_date]
    first_date = returns_data.index[0]
    tickers = list(returns_data.columns)

    weights = pd.DataFrame(0, index=ret_data_filtered.index, columns=ret_data_filtered.columns)
    weights.index = pd.to_datetime(weights.index, format='%Y-%m-%d')
    reb_flag = pd.DataFrame(0, index=weights.index, columns=['reb_flag'])

    alpha = 1 - math.exp(math.log(0.5) / (halflife * annualized))
    span = (2 / alpha) - 1

    for i in range(len(ret_data_filtered)):
        curr_date = ret_data_filtered.index[i]
        curr_date_format = curr_date.strftime('%Y-%m-%d')

        if (weights.index[i].month in rebal_months and weights.index[i-1].month != weights.index[i].month) or i==0:

            if shrink_covar:
                shrink_covar_matrix_model = LedoitWolf().fit(returns_data.loc[first_date:curr_date])
                covar_ann = pd.DataFrame(shrink_covar_matrix_model.covariance_ * annualized, index=tickers, columns=tickers)
                covar_ann.columns = tickers
                covar_ann.index = tickers
            else:
                exp_cov_matrix = returns_data.loc[first_date:curr_date].ewm(span=span).cov(pairwise=True).iloc[-num_of_assets:]
                covar_ann = exp_cov_matrix * annualized
                covar_ann.columns = tickers
                covar_ann.index = tickers

            valid_covar_ann = covar_ann.dropna(how='all')
            valid_covar_ann = valid_covar_ann.dropna(axis=1, how='all')
            valid_num_assets = len(valid_covar_ann)
            valid_tickers = valid_covar_ann.columns

            assets_risk_budget = np.ones([valid_num_assets]) / valid_num_assets
            initial_weights = np.ones([valid_num_assets]) / valid_num_assets

            # print(valid_covar_ann)
            new_wgt = _get_risk_parity_weights(valid_covar_ann.values, assets_risk_budget, initial_weights)
            new_wgt = pd.DataFrame(new_wgt).T
            new_wgt.columns = valid_tickers

            for ticker in valid_tickers:
                weights.at[curr_date_format, ticker] = new_wgt[ticker].values

            reb_flag.loc[curr_date_format] = True

        else:
            weights.iloc[i] = weights.iloc[i-1] * (1+ ret_data_filtered.iloc[i].fillna(0))
            weights_sum = weights.iloc[i].sum()
            weights.iloc[i] /= weights_sum
            reb_flag.iloc[i] = False

    weights = weights.rename(columns={c: c + '_wgt' for c in weights.columns})
    weights = pd.concat([reb_flag, weights], axis=1)

    return weights

def combine_backtest_data(portfolio_names):
    # combined portfolio returns
    all_port_ret = {}

    for portfolio in portfolio_names:
        perf_df = globals()[f"{portfolio}_perf"]
        ret_col = [col for col in perf_df.columns if col.endswith('_port_ret')]
        all_port_ret[portfolio] = perf_df[ret_col]

    df_all_port_ret = pd.concat(all_port_ret.values(), keys=all_port_ret.keys(), axis=1)
    df_all_port_ret.columns = [col[0] for col in df_all_port_ret.columns]

    # combined portfolio weights
    all_port_weights = {}

    for portfolio in portfolio_names:
        wgt_df = globals()[f"{portfolio}_wgt"]
        # wgt_col = [col for col in wgt_df.columns if col.endswith('_wgt')]
        # all_port_weights[portfolio] = wgt_df[wgt_col]
        all_port_weights[portfolio] = wgt_df

    df_all_port_weights = pd.concat(all_port_weights.values(), keys=all_port_weights.keys(), axis=1)

    # combined portfolio backtest daata
    all_port_bt = {}

    for portfolio in portfolio_names:
        bt_df = globals()[f"{portfolio}_perf"]
        all_port_bt[portfolio] = bt_df

    df_all_port_bt = pd.concat(all_port_bt.values(), keys=all_port_bt.keys(), axis=1)
    return df_all_port_ret, df_all_port_weights, df_all_port_bt

Cloning into 'Projects'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 33 (delta 17), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 6.58 MiB | 15.77 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/Projects


# Parameters

Input the following parameters:


*   Start date: format - DD/MM/YYYY
*   End date: format - DD/MM/YYYY
*   Underlying tickers, format - ETF1, ETF2, ETF3, ... | e.g. SPY, EFA, QQQ, RSP, IEMG
*   Benchmark ticker format - ETF1, ETF2, ETF3, ... | e.g. ACWI, URTH, SPY
*   Observation window (days): the number of days after the specified start date used to initialize the optimization process - this ensures enough data is available before the backtest begins (i.e. if lookback window is 365 days and start date is Jan 1 2010, the actual backtest will begin in Jan 1 2011)


❗ If no start and end date is specified, the backtest will begin from the earliest common inception until latest available data




In [7]:
# @title Input data parameters
# Widget for start date
start_date_widget = widgets.DatePicker(
    description='Start Date:',
    # value=parse("2010-01-01").date(),  # Default value
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for end date
end_date_widget = widgets.DatePicker(
    description='End Date:',
    # value=parse("2024-10-31").date(),  # Default value
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for tickers
tickers_widget = widgets.Text(
    value='',
    description='Tickers:',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for benchmark tickers
bm_tickers_widget = widgets.Text(
    value='',
    description='Benchmark Tickers:',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for backtest offset days
backtest_offset_days_widget = widgets.IntText(
    value=365,
    description='Observation window (Days):',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Display the widgets
display(start_date_widget)
display(end_date_widget)
display(tickers_widget)
display(bm_tickers_widget)
display(backtest_offset_days_widget)


# Function to update variables based on widget values
def update_variables(change):
    global start_date, end_date, tickers, bm_tickers, all_tickers, backtest_offset_days

    start_date = start_date_widget.value
    end_date = end_date_widget.value

    tickers = re.split(r'\s*,\s*', tickers_widget.value)
    bm_tickers = re.split(r'\s*,\s*', bm_tickers_widget.value)

    all_tickers = list(set(tickers + bm_tickers))

    backtest_offset_days = backtest_offset_days_widget.value


# Observe widget changes and update variables
start_date_widget.observe(update_variables, names='value')
end_date_widget.observe(update_variables, names='value')
tickers_widget.observe(update_variables, names='value')
bm_tickers_widget.observe(update_variables, names='value')
backtest_offset_days_widget.observe(update_variables, names='value')

# Initial variable update
update_variables(None)

DatePicker(value=None, description='Start Date:', layout=Layout(width='300px'), style=DescriptionStyle(descrip…

DatePicker(value=None, description='End Date:', layout=Layout(width='300px'), style=DescriptionStyle(descripti…

Text(value='', description='Tickers:', layout=Layout(width='300px'), style=DescriptionStyle(description_width=…

Text(value='', description='Benchmark Tickers:', layout=Layout(width='300px'), style=DescriptionStyle(descript…

IntText(value=365, description='Observation window (Days):', layout=Layout(width='300px'), style=DescriptionSt…

In [9]:
# @title Download data
# df_all = yf.download(all_tickers, start=start_date, end=end_date)['Adj Close']
if start_date is None and end_date is None:
    df_all = yf.download(all_tickers, period='max')['Adj Close']
else:
    df_all = yf.download(all_tickers, start=start_date, end=end_date)['Adj Close']

df_all.index = df_all.index.tz_localize(None)

ret_all = df_all.pct_change().dropna()

ret_bm = ret_all[bm_tickers]

earliest_start_date = ret_all.index.min()
start_date_timestamp = pd.Timestamp(start_date)

# Compare earliest_start_date with start_date_timestamp
if earliest_start_date > start_date_timestamp:
    print('\n', 'Earliest start date: ', earliest_start_date)
else:
    pass

[*********************100%***********************]  4 of 4 completed


 Earliest start date:  2020-01-03 00:00:00


# Constituent performance

In [10]:
# @title Calculate constituent performance
constituent_summary = pc.performance_summary_constituents(ret_all, start_date=start_date, end_date=end_date, frequency='daily')
constituent_performance_button = widgets.Button(description="Generate Constituent Performance", layout=widgets.Layout(width='auto'))

# Use an Output widget to display the results
out = widgets.Output()

def run_constituent_performance(b):
    with out:
        clear_output(wait=True)  # Clear previous output within the Output widget
        constituent_summary = pc.performance_summary_constituents(
            ret_all, start_date=start_date, end_date=end_date, frequency='daily')
        display(constituent_summary)
        return constituent_summary

constituent_performance_button.on_click(run_constituent_performance)

# Display the button and the Output widget
display(constituent_performance_button, out)

Button(description='Generate Constituent Performance', layout=Layout(width='auto'), style=ButtonStyle())

Output()

In [11]:
# @title Calculate constituent calendar year returns
yearly_df = pc.constituents_calendar_year_returns(ret_all, frequency='daily')
yearly_returns_button = widgets.Button(description="Generate Calendar Year Returns", layout=widgets.Layout(width='auto'))
out = widgets.Output()  # Output widget to display the returns

def generate_yearly_returns(b):

  with out:
    clear_output(wait=True)
    yearly_df = pc.constituents_calendar_year_returns(ret_all, frequency='daily')
    display(yearly_df.pipe(pc.apply_style_heatmap_ret))
    return yearly_df

yearly_returns_button.on_click(generate_yearly_returns)
print('Date range: ', ret_all.index.min(), ' - ', ret_all.index.max(), '\n')
display(yearly_returns_button, out)

Date range:  2020-01-03 00:00:00  -  2024-10-30 00:00:00 



Button(description='Generate Calendar Year Returns', layout=Layout(width='auto'), style=ButtonStyle())

Output()

In [12]:
# @title Correlation matrix
def generate_correlation(b):
       with out:
           clear_output(wait=True)
           covar = pc.exp_covar(ret_all, halflife=3.5, annualized=261)
           corr = pd.DataFrame(pc.cov2corr(covar),index=covar.columns, columns=covar.columns)
           display(corr.pipe(pc.apply_style_heatmap).format('{:.2}'))


correlation_button = widgets.Button(description="Generate Correlation Matrix", layout=widgets.Layout(width='auto'))
out = widgets.Output()
correlation_button.on_click(generate_correlation)
display(correlation_button, out)

Button(description='Generate Correlation Matrix', layout=Layout(width='auto'), style=ButtonStyle())

Output()

In [14]:
# @title Risk/return scatter plot
period_widget = widgets.IntText(
    description='Period (year):', disabled=False)
period_widget.value = 0 # Set initial value to 0
display(period_widget)

data_years = (ret_all.index[-1] - ret_all.index[0]).days / 365.25  # Total years of data

def update_plot(change):
    clear_output(wait=True)
    try:
        period = period_widget.value
        if period > data_years:
            print(f"Not enough data for {period} years. Available data is for {data_years:.1f} years.")
        else:
            pc.plot_return_risk_scatter_year(ret_all, freq='daily', periods=[period], show_data=True)
    except ValueError:
        print("Invalid input. Please enter an integer value for the period.")

period_widget.observe(update_plot, names='value')


# Create a button widget
return_risk_button = widgets.Button(description="Generate Return Risk Chart", layout=widgets.Layout(width='auto'))

# Function to execute when the button is clicked
def on_return_risk_button_clicked(b):
    with out:
        clear_output(wait=True)
        period = period_widget.value  # Get current value from the widget
        pc.plot_return_risk_scatter_year(ret_all, freq='daily', periods=[period], show_data=True)

# Use an Output widget to display the chart
out = widgets.Output()

# Attach the function to the button's on_click event
return_risk_button.on_click(on_return_risk_button_clicked)

# Display the button and the Output widget
display(return_risk_button, out)

,MOAT,QQQ,RSP,SPY
3Y_Annualized return,10.32%,11.45%,7.67%,11.78%
3Y_Annualized volatility,19.39%,23.92%,17.14%,17.69%
3Y_Sharpe ratio,0.532,0.479,0.448,0.666


# Portfolio configuration & parameters

In [15]:
# @title Input portfolio parameters
# @markdown If 'Allocation' = 'Custom weight' , weight input field will be visible
def create_portfolio_widgets(portfolio_number):
    """Creates and displays widgets for a single portfolio."""
    display(widgets.HTML(f"<h3>Portfolio {portfolio_number}</h3>"))

    tickers_widget = widgets.Text(
        value='',
        description='Tickers:',
        disabled=False
    )
    allocation_type_widget = widgets.Dropdown(
        options=['Max sharpe', 'Min vol', 'Risk parity', 'Equal weight', 'Custom weight'],
        value='Max sharpe',  # Set default value based on portfolio_number or user preference
        description='Allocation:',
        disabled=False,
    )
    name_widget = widgets.Text(
        value=f'P{portfolio_number}',  # Set default name based on portfolio_number or user preference
        description='Name:',
        disabled=False
    )
    min_weight_widget = widgets.FloatText(
        value=0.0,
        description='Min Weight:',
        disabled=False
    )
    max_weight_widget = widgets.FloatText(
        value=1.0,
        description='Max Weight:',
        disabled=False
    )

    custom_weight_widget = widgets.Text(value='',  # Initial value can be empty
        description='Input Weight:',
        disabled=False,
        layout=widgets.Layout(visibility='hidden') # Initially hidden
    )

    def toggle_custom_weight_visibility(change):
        if allocation_type_widget.value == 'Custom weight':
            custom_weight_widget.layout.visibility = 'visible'
        else:
            custom_weight_widget.layout.visibility = 'hidden'

    # Observe the allocation_type_widget for changes
    allocation_type_widget.observe(toggle_custom_weight_visibility, names='value')

    display(tickers_widget)
    display(allocation_type_widget)
    display(name_widget)
    display(min_weight_widget)
    display(max_weight_widget)

    # Display the custom_weight_widget
    display(custom_weight_widget)


    return {
        'tickers': tickers_widget,
        'allocation_type': allocation_type_widget,
        'name': name_widget,
        'min_weight': min_weight_widget,
        'max_weight': max_weight_widget,
        'custom_weight': custom_weight_widget  # Add custom_weight_widget to the dictionary
    }

# Create widgets for 5 portfolios
portfolio_widgets = {}
for i in range(1, 6):
    portfolio_widgets[i] = create_portfolio_widgets(i)


HTML(value='<h3>Portfolio 1</h3>')

Text(value='', description='Tickers:')

Dropdown(description='Allocation:', options=('Max sharpe', 'Min vol', 'Risk parity', 'Equal weight', 'Custom w…

Text(value='P1', description='Name:')

FloatText(value=0.0, description='Min Weight:')

...

Dropdown(description='Allocation:', options=('Max sharpe', 'Min vol', 'Risk parity', 'Equal weight', 'Custom w…

Text(value='P5', description='Name:')

FloatText(value=0.0, description='Min Weight:')

FloatText(value=1.0, description='Max Weight:')

Text(value='', description='Input Weight:', layout=Layout(visibility='hidden'))

In [16]:
# @title Confirm portfolio parameters
def get_portfolio_specs():
    portfolio_specs = {}
    for i in range(1, 6):
        portfolio_specs[i] = {
            'tickers': re.split(r'\s*,\s*', portfolio_widgets[i]['tickers'].value),
            'allocation_type': portfolio_widgets[i]['allocation_type'].value,
            'name': portfolio_widgets[i]['name'].value,
            'min_weight': float(portfolio_widgets[i]['min_weight'].value),
            'max_weight': float(portfolio_widgets[i]['max_weight'].value),
            'custom_weight': portfolio_widgets[i]['custom_weight'].value  # Get the value of the custom weight widget
        }

        if portfolio_specs[i]['allocation_type'] == 'Custom weight':
            try:
                portfolio_specs[i]['custom_weight'] = [float(x) for x in portfolio_specs[i]['custom_weight'].split(',')]
            except ValueError:
                print("Invalid custom weight input. Please enter comma-separated values and ensure values sum to 1 (e.g 0.2,0.3,0.1,0.2,0.2) 🙏")
                # You might want to handle the error more gracefully here.

    return portfolio_specs

# Function to execute on button click
def on_confirm_button_clicked(b):
  display(df_portfolio_specs)
  print('\n')
  print("Portfolio specifications confirmed! ✅")
  # You can add any further actions you want to perform here
  # after confirmation, such as starting the optimization.


# Define constraints function
def create_constraints(min_weight, max_weight):
    return ({
        'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1
    }, {
        'type': 'ineq', 'fun': lambda weights: weights - min_weight
    }, {
        'type': 'ineq', 'fun': lambda weights: max_weight - weights
    })


def create_custom_weight_dict(portfolio_specs, portfolio_number):
  # Creates a dictionary with ticker as key and custom weight as value.
  portfolio = portfolio_specs[portfolio_number]
  tickers = portfolio['tickers']
  custom_weights = portfolio['custom_weight']

  if len(tickers) != len(custom_weights):
    raise ValueError("Number of tickers and custom weights must be equal.")

  custom_weight_dict = dict(zip(tickers, custom_weights))
  return custom_weight_dict

def check_custom_weight_sum(portfolio_specs):

  for portfolio_number, portfolio_data in portfolio_specs.items():
      if portfolio_data['allocation_type'] == 'Custom weight':
          custom_weights = portfolio_data['custom_weight']
          if np.isclose(np.sum(custom_weights), 1.0):
              pass
          else:
              print(f"Portfolio {portfolio_number}: Custom weights do not sum to 1. Please amend 🙏.")


portfolio_specs = get_portfolio_specs()
df_portfolio_specs = pd.DataFrame(portfolio_specs).T

# Create a confirmation button
confirm_button = widgets.Button(description="Confirm Portfolio Specs", layout=widgets.Layout(width='auto'))

# Attach the function to the button's on_click event
confirm_button.on_click(on_confirm_button_clicked)

# Display the button
display(confirm_button)

# Call the function to check the custom weight sum
check_custom_weight_sum(portfolio_specs)

Button(description='Confirm Portfolio Specs', layout=Layout(width='auto'), style=ButtonStyle())

,tickers,allocation_type,name,min_weight,max_weight,custom_weight
1,"[RSP, QQQ, MOAT]",Max sharpe,P1,0.05,0.5,
2,"[RSP, QQQ, MOAT]",Min vol,P2,0.05,0.5,
3,"[RSP, QQQ, MOAT]",Risk parity,P3,0.0,1.0,
4,"[RSP, QQQ, MOAT]",Equal weight,P4,0.0,1.0,
5,"[RSP, QQQ, MOAT]",Custom weight,P5,0.0,1.0,"[0.2, 0.4, 0.4]"




Portfolio specifications confirmed! ✅


In [17]:
# @title Run optimisation

# Create a progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=3, # Total number of portfolios
    description='Optimising:',
    bar_style='',
    style={'bar_color': 'navy'},
    orientation='horizontal'
)

# Display the progress bar
display(progress_bar)

for i in range(1, 6):
    portfolio = portfolio_specs[i]
    portfolio_name = portfolio['name']
    tickers = portfolio['tickers']
    allocation_type = portfolio['allocation_type']
    min_weight = portfolio['min_weight']
    max_weight = portfolio['max_weight']
    custom_weight = portfolio['max_weight']

    # Create constraints for each portfolio based on the min and max weight
    constraints = create_constraints(min_weight, max_weight)

    # Calculate weights based on allocation type:
    if allocation_type == 'Max sharpe':
        globals()[f"{portfolio_name}_wgt"] = pc.max_sharpe_rebal_wgt(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            constraints_dict=constraints,
            shrink_covar=False,
            rebal_months=[1],
            annualized=261,
            halflife=3.5
        )
    elif allocation_type == 'Min vol':
        globals()[f"{portfolio_name}_wgt"] = pc.min_vol_rebal_wgt(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            constraints_dict=constraints,
            shrink_covar=False,
            rebal_months=[1],
            annualized=261,
            halflife=3.5
        )
    elif allocation_type == 'Risk parity':
        globals()[f"{portfolio_name}_wgt"] = rebal_wgt_riskparity(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            rebal_months=[1],
            halflife=3.5,
            annualized=261,
            shrink_covar=False
        )

    elif allocation_type == 'Equal weight':
        globals()[f"{portfolio_name}_wgt"] = pc.equal_weight_rebal_wgt(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            rebal_months=[1],
        )

    elif allocation_type == 'Custom weight':
        globals()[f"{portfolio_name}_wgt"] = pc.fixed_weight_rebal_wgt(
          ret_all[tickers],
          start_date=earliest_start_date + timedelta(days=backtest_offset_days),
          end_date=end_date,
          fixed_weights=create_custom_weight_dict(portfolio_specs, i),
          rebal_months=[1],
    )

    # Calculate performance:
    globals()[f"{portfolio_name}_perf"] = pc.calc_port_perf(ret_all.loc[earliest_start_date + timedelta(days=backtest_offset_days):end_date][tickers],
                                                            globals()[f"{portfolio_name}_wgt"], name_prefix=portfolio_name)

    # Update the progress bar
    progress_bar.value += 1

clear_output(wait=True) # Clear the output after completion
print("Optimisation complete!")


Optimisation complete!


In [18]:
# @title Historical weights at each point of rebalance
def show_historical_weights(b):
       clear_output(wait=True) # Clear previous output
       for i in range(1, 6):
           portfolio = portfolio_specs[i]
           portfolio_name = portfolio['name']
           print(portfolio_name)
           pc.display_rebal_wgt(globals()[f"{portfolio_name}_wgt"])
           print('\n')

hist_weights_button = widgets.Button(description="Show Historical Weights", layout=widgets.Layout(width='auto'))
hist_weights_button.on_click(show_historical_weights)
display(hist_weights_button)


P1


,reb_flag,RSP_wgt,QQQ_wgt,MOAT_wgt
Date,,,,
2021-01-04 00:00:00,100.00%,5.00%,50.00%,45.00%
2022-01-03 00:00:00,100.00%,19.17%,50.00%,30.83%
2023-01-03 00:00:00,100.00%,50.00%,44.29%,5.71%
2024-01-02 00:00:00,100.00%,5.00%,50.00%,45.00%




P2


,reb_flag,RSP_wgt,QQQ_wgt,MOAT_wgt
Date,,,,
2021-01-04 00:00:00,100.00%,5.00%,45.00%,50.00%
2022-01-03 00:00:00,100.00%,11.18%,38.82%,50.00%
2023-01-03 00:00:00,100.00%,50.00%,5.00%,45.00%
2024-01-02 00:00:00,100.00%,50.00%,7.46%,42.54%




P3


,reb_flag,RSP_wgt,QQQ_wgt,MOAT_wgt
Date,,,,
2021-01-04 00:00:00,100.00%,31.39%,34.56%,34.05%
2022-01-03 00:00:00,100.00%,31.92%,33.41%,34.67%
2023-01-03 00:00:00,100.00%,34.78%,31.06%,34.16%
2024-01-02 00:00:00,100.00%,35.23%,31.05%,33.73%




P4


,reb_flag,RSP_wgt,QQQ_wgt,MOAT_wgt
Date,,,,
2021-01-04 00:00:00,100.00%,33.33%,33.33%,33.33%
2022-01-03 00:00:00,100.00%,33.33%,33.33%,33.33%
2023-01-03 00:00:00,100.00%,33.33%,33.33%,33.33%
2024-01-02 00:00:00,100.00%,33.33%,33.33%,33.33%




P5


,reb_flag,RSP_wgt,QQQ_wgt,MOAT_wgt
Date,,,,
2021-01-04 00:00:00,100.00%,20.00%,40.00%,40.00%
2022-01-03 00:00:00,100.00%,20.00%,40.00%,40.00%
2023-01-03 00:00:00,100.00%,20.00%,40.00%,40.00%
2024-01-02 00:00:00,100.00%,20.00%,40.00%,40.00%


# Key Data

In [39]:
benchmark_ticker

NameError: name 'benchmark_ticker' is not defined

'SPY'

In [44]:
# @title Generate key data

########################################################################################################################

port_names = []  # Initialize an empty list to store names

# Iterate through the portfolio specifications
for portfolio_number, portfolio_data in portfolio_specs.items():
    # Extract the 'name' attribute and append it to the list
    port_names.append(portfolio_data['name'])

all_port_ret, all_port_wgt, all_port_bt = combine_backtest_data(port_names)
ret_bm_filtered = ret_bm.loc[earliest_start_date + timedelta(days=backtest_offset_days):]
ret_bm_filtered.iloc[0] = 0
all_port_ret = pd.concat([all_port_ret, ret_bm], axis=1).dropna()

########################################################################################################################


perf_summary = pc.performance_summary(all_port_ret[port_names], all_port_wgt, all_port_ret[bm_tickers[0]], benchmark_returns=ret_bm,
                                    start_date=earliest_start_date + timedelta(days=backtest_offset_days), end_date=end_date, frequency='daily', rebal_per_year=1)

relative_perf = pc.calculate_relative_per(all_port_ret[port_names], all_port_ret[bm_tickers[0]])
df_ports_alloc_latest = pc.compare_port_alloc(all_port_wgt, port_names, latest_data=True)
df_ports_alloc_avg = pc.compare_port_alloc(all_port_wgt, port_names, latest_data=False)

port_turnover_df = pd.DataFrame()

for portfolio in port_names:
    turnover = pc.portfolio_turnover(all_port_wgt[portfolio])
    port_turnover_df = pd.concat([port_turnover_df, turnover],axis=1)

port_turnover_df.columns = port_names


def generate_key_data_func(b):
       clear_output(wait=True)  # Clear previous output
       generate_key_data() # Assuming generate_key_data is already defined


key_data_button = widgets.Button(description="Generate Key Data")
key_data_button.on_click(generate_key_data_func)
display(key_data_button)


def generate_key_data():

    # Create output widgets for each tab
    out_summary = widgets.Output()
    out_yearly_returns = widgets.Output()
    out_cumulative_returns = widgets.Output()
    out_returns_distribution = widgets.Output()
    out_latest_allocation = widgets.Output()
    out_average_allocation = widgets.Output()
    out_portfolio_allocation = widgets.Output()
    out_portfolio_drawdown = widgets.Output()
    out_correlation = widgets.Output()

    # Populate output widgets with data
    with out_summary:
        display(perf_summary)
    with out_yearly_returns:
        display(pc.apply_style_heatmap_ret(relative_perf, subset=relative_perf.columns[1:]))
    with out_cumulative_returns:
        pc.plot_cumulative_returns(all_port_ret, show_data=True)
    with out_returns_distribution:
        pc.plot_returns_distribution_boxplot(all_port_ret)
    with out_latest_allocation:
        display(df_ports_alloc_latest.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))
    with out_average_allocation:
        display(df_ports_alloc_avg.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))
    with out_portfolio_allocation:
        for port in port_names:
            print(port)
            pc.plot_weights_and_turnover(all_port_wgt[port], show_data=True, show_turnover=False, show_rebal=True)
    with out_portfolio_drawdown:
        pc.plot_drawdowns(all_port_ret, show_data=True)
    with out_correlation:
        pc.plot_correlation_heatmap(all_port_ret)


    # Create tabs and assign output widgets
    tab = widgets.Tab()
    tab.children = [out_summary, out_yearly_returns, out_cumulative_returns, out_returns_distribution,
                    out_latest_allocation, out_average_allocation, out_portfolio_allocation,
                    out_portfolio_drawdown, out_correlation]
    tab.set_title(0, 'Key Summary')
    tab.set_title(1, 'Relative Yearly Returns')
    tab.set_title(2, 'Cumulative Returns')
    tab.set_title(3, 'Returns Distribution')
    tab.set_title(4, 'Latest Allocation')
    tab.set_title(5, 'Average Allocation')
    tab.set_title(6, 'Portfolio Allocation')
    tab.set_title(7, 'Portfolio Drawdown')
    tab.set_title(8, 'Correlation')

    display(tab)


In [20]:
pc.constituents_calendar_year_returns(all_port_ret, frequency='daily')

,P1,P2,P3,P4,P5,SPY
2021,0.276698,0.274834,0.286212,0.286879,0.281649,0.305055
2022,-0.226433,-0.207053,-0.192929,-0.192552,-0.207896,-0.181754
2023,0.332720,0.238017,0.327912,0.335919,0.375759,0.261758
2024,0.174154,0.141564,0.159675,0.161448,0.165413,0.231784


In [78]:
for i in port_names:
  portfolio_returns = all_port_ret[[i]]
  monthly_performance = monthly_performance_table(portfolio_returns)
  print(i)
  display(monthly_performance.pipe(pc.apply_style_heatmap_ret))
  print('\n')

P1


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Date,,,,,,,,,,,,
2021,1.06%,2.96%,3.95%,5.05%,0.25%,3.44%,2.38%,2.81%,-4.97%,5.85%,-0.64%,3.01%
2022,-5.86%,-2.70%,3.28%,-10.27%,-0.53%,-8.59%,10.89%,-4.75%,-10.05%,6.07%,6.85%,-6.92%
2023,9.31%,-1.99%,4.21%,0.47%,2.06%,6.90%,3.73%,-2.36%,-5.10%,-3.11%,10.07%,6.24%
2024,0.15%,4.72%,2.46%,-4.66%,3.90%,3.34%,1.55%,2.60%,2.20%,0.24%,nan%,nan%




P2


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Date,,,,,,,,,,,,
2021,1.00%,3.27%,4.17%,4.96%,0.39%,3.18%,2.33%,2.67%,-4.90%,5.64%,-0.91%,3.19%
2022,-4.99%,-2.35%,2.86%,-9.70%,-0.45%,-8.30%,10.73%,-4.87%,-10.05%,6.10%,7.39%,-6.63%
2023,9.46%,-3.00%,2.24%,0.69%,-1.20%,7.06%,3.94%,-3.34%,-5.25%,-4.29%,9.61%,7.23%
2024,-1.06%,4.18%,3.87%,-4.85%,2.49%,0.25%,4.32%,3.19%,2.06%,-0.73%,nan%,nan%




P3


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Date,,,,,,,,,,,,
2021,0.93%,3.92%,4.61%,4.91%,0.77%,2.38%,2.06%,2.64%,-4.62%,5.63%,-1.29%,3.97%
2022,-5.11%,-2.12%,2.92%,-9.15%,-0.15%,-8.61%,10.31%,-4.54%,-9.85%,6.88%,7.09%,-6.26%
2023,9.97%,-2.26%,4.37%,0.64%,1.56%,6.81%,3.93%,-2.76%,-5.20%,-3.59%,10.01%,6.71%
2024,-0.43%,4.47%,3.16%,-4.73%,3.42%,1.93%,2.66%,2.67%,2.20%,-0.10%,nan%,nan%




P4


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Date,,,,,,,,,,,,
2021,0.91%,4.00%,4.66%,4.90%,0.81%,2.29%,2.03%,2.63%,-4.59%,5.61%,-1.34%,4.04%
2022,-5.14%,-2.12%,2.93%,-9.13%,-0.13%,-8.64%,10.28%,-4.52%,-9.84%,6.93%,7.06%,-6.24%
2023,10.02%,-2.19%,4.56%,0.64%,1.79%,6.78%,3.93%,-2.72%,-5.20%,-3.54%,10.04%,6.67%
2024,-0.37%,4.50%,3.09%,-4.72%,3.50%,2.09%,2.51%,2.63%,2.21%,-0.04%,nan%,nan%




P5


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Date,,,,,,,,,,,,
2021,0.97%,3.58%,4.38%,4.95%,0.58%,2.77%,2.18%,2.68%,-4.76%,5.67%,-1.08%,3.60%
2022,-5.30%,-2.37%,2.99%,-9.67%,-0.37%,-8.48%,10.63%,-4.73%,-9.99%,6.35%,7.15%,-6.58%
2023,10.54%,-1.96%,5.60%,0.69%,2.80%,6.64%,4.00%,-2.63%,-5.22%,-3.44%,10.19%,6.64%
2024,-0.25%,4.59%,2.82%,-4.70%,3.63%,2.61%,2.13%,2.65%,2.20%,0.05%,nan%,nan%


In [68]:
plot_return_risk_scatter_maxrange(all_port_ret)

,P1,P2,P3,P4,P5,SPY
Annualized return,12.52%,10.15%,13.56%,13.82%,14.11%,14.72%
Annualized volatility,18.98%,18.52%,18.16%,18.16%,18.72%,16.86%
Sharpe ratio,0.66,0.548,0.747,0.761,0.754,0.873


In [51]:
pc.plot_rolling_correlation(all_port_ret[port_names], benchmark=all_port_ret[bm_tickers[0]], window=261)

In [75]:
pc.plot_rolling_volatility(all_port_ret)

# Export data to excel

In [27]:
# @title Input file name and export key data to excel
# Create a text input widget for the file path
file_path_widget = widgets.Text(
    value='Portfolio_Visualizer_Output.xlsx',
    description='File Name:',
    disabled=False
)

# Display the widget
display(file_path_widget)

# Function to save the data to Excel
def save_to_excel(file_path):
    with pd.ExcelWriter(file_path) as writer:
      constituent_summary.to_excel(writer, sheet_name='Constituents summary', index=True)
      yearly_df.to_excel(writer, sheet_name='Constituents calendar year returns', index=True)
      perf_summary.to_excel(writer, sheet_name='Perf summary', index=True)
      # yearly_df.filter(like='_Annual Return').to_excel(writer, sheet_name='Calendar year returns', index=True)
      df_ports_alloc_latest.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Latest alloc', index=True)
      df_ports_alloc_avg.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Average alloc', index=True)

      # country_breakdown_all.pipe(apply_style_heatmap).pipe(codebase.apply_2dp_percentage).to_excel(writer, sheet_name='Country', index=True)
      # sector_breakdown_all.pipe(apply_style_heatmap).pipe(codebase.apply_2dp_percentage).to_excel(writer, sheet_name='Sector', index=True)
      # country_diff_df.to_excel(writer, sheet_name='Country Relative', index=True)
      # sector_diff_df.to_excel(writer, sheet_name='Sector Relative', index=True)
      # port_turnover_df.to_excel(writer, sheet_name='Turnover', index=True)
      # df_factor_exp.to_excel(writer, sheet_name='ETF Factor MSCI', index=True)
      # df_factor_ff_exp.to_excel(writer, sheet_name='ETF Factor FF', index=True)
      # df_factor_port_exp.to_excel(writer, sheet_name='Portfolio Factor MSCI', index=True)
      # df_factor_ff_port_exp.to_excel(writer, sheet_name='Portfolio Factor FF', index=True)
      # exposure_diff.to_excel(writer, sheet_name='Relative Factor MSCI', index=True)
      # exposure_diff_ff.to_excel(writer, sheet_name='Relative Factor FF', index=True)

      for n in port_names:
          all_port_wgt[n].to_excel(writer, sheet_name=n+'_hist_wgt', index=True)

      print(f"Excel file saved to: Projects/Projects/{file_path}")

# Create a button to trigger the save function
save_button = widgets.Button(description="Save as Excel")

# Define the button's on_click behavior
def on_save_button_clicked(b):
    save_to_excel(file_path_widget.value)

save_button.on_click(on_save_button_clicked)

# Display the button
display(save_button)

Text(value='Portfolio_Visualizer_Output.xlsx', description='File Name:')

Button(description='Save as Excel', style=ButtonStyle())